## Content

- **XGBoost**
    

- **LightGBM**

- **Stacking**


- **Cascading**



## XGBoost

<center><img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/062/297/original/z.png?1705487607' width=700></center>

Sklearn's GBDT implentation is not the best implementation.

Recall how much time it took (>30 mins)
- to do hyperparam tuning/ training single model using sklearn GBDT

This is where **XGBoost** steps in


XGBoost provides **optimized** **implementation** of GBDT
- which helps in reducing model training process.


Let's see how it does that

### What optimization does XGBoost provides ?

#### Parallelization of features selection

While building Decision Trees,
- there will be n features to consider while splitting the node.

<br>

The computation of Information Gain(s) of the features is done in parallel
- which helps in reducing the training time

<center><img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/062/298/original/z.png?1705487635' width=700></center>

#### Parallelization in building DT

While building a DT,
- both subtrees (left and right) can be build in parallel
- as there is no dependency between them.

which helps in making the process faster and efficient.

<center><img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/062/299/original/z.png?1705487664' width=700></center>

#### Optimizing thresholding in numerical feature

In conventional DTs,
- While finding the threshold for numerical feature
    - all the numerical values are tested to find one with maximum information gain.

Which makes this whole process slow.



XGBoost optimizes this by using **histogram based binning**

**What's histogram based binning ?**

- In simple terms, it creates discrete bins (percentile  binning) using these continuos values
- And then selects threshold using the bins instead of trying every single value.

i.e. it uses approximation.

- which helps in making the process faster.

<center><img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/062/300/original/z.png?1705487688' width=700></center>

If you want to dive deep into it,

here's the link to the research paper: https://www.kdd.org/kdd2016/papers/files/rfp0697-chenAemb.pdf
- published in 2016

Let's look into the numerous hyperparams provided by XGBoost

### Hyperparameters

XGBoost doc: https://xgboost.readthedocs.io/en/stable/python/python_api.html#xgboost.XGBClassifier

<center><img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/062/301/original/z.png?1705487728' width=700></center>



<center><img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/062/302/original/z.png?1705487755' width=700></center>

<center><img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/062/303/original/z.png?1705487781' width=700></center>

1. **Eta**: or the learning rate is the shrinking/regularization term

2. **min_split_loss** specify the minimum Information Gain which you want for further split.

    - If the min_split_loss value of the model is increased,
    - the splitting stops if the min_split_loss is not met.
    
Due to this the depth decreases resulting in shallow tress.

- Hence,results in the underfitting of the model.


3. **max_depth**, this parameter is used to set the depth of the base learners  

4. **min_child_weight**: you can increase the weight of the child due to which the splitting stops if the required threshold is not met

5. **subsample**. is nothing but the row sampling rate.





It provides various levels of column sampling

i.e.
- column sampling ratio for each tree
- column sampling ratio for each level
- column sampling ratio for each split

It even provides L1 and L2 regularization for weights ($γ_m$)

Now that we are aware of the hyperparams,

let's give them a try

### Code walkthrough

In [ ]:
import pickle

!gdown 171Yoe_GSapyrmOnD9oBzHWNOD_OnQs0F
!gdown 1hnIlTPW3AMeB69EbeaXCRIrpMVT1Vwmc
!gdown 1nZtB_RtxMg_MgoRczb8UWQX-AEK_l3qE
!gdown 1zLDUErwKdmF-RacOyHEuI_z_46LssQtP


with open('X_train.pickle', 'rb') as handle:
    X_train = pickle.load(handle)

with open('X_test.pickle', 'rb') as handle:
    X_test = pickle.load(handle)

with open('Y_train.pickle', 'rb') as handle:
    y_train = pickle.load(handle)

with open('Y_test.pickle', 'rb') as handle:
    y_test = pickle.load(handle)

Downloading...
From: https://drive.google.com/uc?id=171Yoe_GSapyrmOnD9oBzHWNOD_OnQs0F
To: /content/Y_test.pickle
100% 31.7k/31.7k [00:00<00:00, 83.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1hnIlTPW3AMeB69EbeaXCRIrpMVT1Vwmc
To: /content/X_test.pickle
100% 253k/253k [00:00<00:00, 129MB/s]
Downloading...
From: https://drive.google.com/uc?id=1nZtB_RtxMg_MgoRczb8UWQX-AEK_l3qE
To: /content/Y_train.pickle
100% 126k/126k [00:00<00:00, 130MB/s]
Downloading...
From: https://drive.google.com/uc?id=1zLDUErwKdmF-RacOyHEuI_z_46LssQtP
To: /content/X_train.pickle
100% 1.01M/1.01M [00:00<00:00, 152MB/s]


In [ ]:

from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold

import datetime as dt

params = {
        "n_estimators": [50,100,150,200],
        "max_depth" : [3, 4, 5, 7],
        "learning_rate": [0.1, 0.2, 0.3],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        }
xgb = XGBClassifier(objective='multi:softmax', num_class=20, silent=True)




In [ ]:

random_search = RandomizedSearchCV(xgb,
                                   param_distributions=params,
                                   n_iter=10,
                                   scoring='accuracy',
                                   n_jobs=-1,
                                   cv=3,
                                   verbose=2)


start = dt.datetime.now()
random_search.fit(X_train, y_train)
end = dt.datetime.now()


Fitting 3 folds for each of 10 candidates, totalling 30 fits
[03:38:03] WARNING: ../src/learner.cc:767: 
Parameters: { "max_leaf_nodes", "silent" } are not used.



In [ ]:
res = random_search.cv_results_

for i in range(len(res["params"])):
  print(f"Parameters:{res['params'][i]} Mean_score: {res['mean_test_score'][i]} Rank: {res['rank_test_score'][i]}")


Parameters:{'subsample': 0.6, 'n_estimators': 200, 'max_leaf_nodes': 80, 'max_depth': 5, 'learning_rate': 0.3, 'colsample_bytree': 0.6} Mean_score: 0.958523592085236 Rank: 4
Parameters:{'subsample': 0.8, 'n_estimators': 150, 'max_leaf_nodes': 40, 'max_depth': 7, 'learning_rate': 0.1, 'colsample_bytree': 0.8} Mean_score: 0.9615677321156774 Rank: 2
Parameters:{'subsample': 1.0, 'n_estimators': 200, 'max_leaf_nodes': 80, 'max_depth': 7, 'learning_rate': 0.2, 'colsample_bytree': 0.8} Mean_score: 0.9650558092338914 Rank: 1
Parameters:{'subsample': 0.8, 'n_estimators': 150, 'max_leaf_nodes': 40, 'max_depth': 5, 'learning_rate': 0.1, 'colsample_bytree': 0.8} Mean_score: 0.9540208016235413 Rank: 6
Parameters:{'subsample': 0.8, 'n_estimators': 100, 'max_leaf_nodes': 40, 'max_depth': 5, 'learning_rate': 0.2, 'colsample_bytree': 0.6} Mean_score: 0.9534500253678336 Rank: 7
Parameters:{'subsample': 0.6, 'n_estimators': 150, 'max_leaf_nodes': 20, 'max_depth': 5, 'learning_rate': 0.2, 'colsample_bytr

In [ ]:
print(f"Time taken for fits : {end - start}")

Time taken for fits : 0:09:44.158693


In [ ]:
print(random_search.best_estimator_)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaf_nodes=80,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=200, n_jobs=None,
              num_class=20, num_parallel_tree=None, ...)


In [ ]:
xgb = random_search.best_estimator_

xgb.fit(X_train, y_train)

print("Model acc",xgb.score(X_test, y_test))



[03:45:08] WARNING: ../src/learner.cc:767: 
Parameters: { "max_leaf_nodes", "silent" } are not used.

Model acc 0.9807253360385493


Notice that
- It took us 10 mins to hyperparam tune XGB vs > 30 mins in sklearn GBDT


## LightGBM

Research paper link: https://proceedings.neurips.cc/paper_files/paper/2017/file/6449f44a102fde848669bdd9eb6b76fa-Paper.pdf

- published in 2017
- by Microsoft Research

LightGBM is another implementation of GBDT
- uses insane optimization to make the training efficient.

Surprisingly, it is **faster than XGBoost**

### What makes LightGBM faster?

```


How can we make an algo train faster?

a. reduce number of datapoints
b. reduce number of features
c. Both
d. None of the aboe

Ans: c. Both

Reducing the number of datapoint or features will help algo train faster.
```

This is what LightGBM does with its optimization

 Let's look into that

#### GOSS (Gradient-based One-Side Sampling)

Say, we are training the $m^{th}$ model of GBDT

<center><img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/062/304/original/z.png?1705487819' width=800></center>

During training,
- there will be lot of datapoints with small residual (pseudo residual)

So, what LightGBM does is
- it drops all the points from training where the error is very small.

Intuitively,
- It is doing **smart sampling** by
    - reducing the size of training data
    - which make the training process faster.

#### But, why does one side sampling means ?

<center><img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/062/305/original/z.png?1705487847' width=800></center>

We are sampling the points from one side of the distribution
- Hence, the name Gradient based One Side sampling



Moving on to next optimization

#### EFB (Exclusive Feature Bundling)

As the name suggests,
- it scans through all the features
- and tries to find feature pairs which are exclusive

#### What does exclusive feature mean ?

Say, we have an OHE encoded categorical feature

For example:
- Gender when encoded using OHE gave us two features Male & Female.

<center><img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/062/306/original/z.png?1705487872' width=800></center>

Notice that
- when value of Male is 1
- Female is 0

i.e. when $f_3$ (Male) is one, other featue ($f_5$) is zero
- The behaviour is called exclusivity.

#### But, it is obvious that when Male feature value is 1, female feature value will be zero.

Yes.
But
- a problem has numerous features
- and we can't check manually which features are exclusive

There can be a case where two categorical features
- say, for example:
    - binary feature: whether person likes football ?
    - and another feature: whether person likes swimming ?

There can be chance that this pair of feature is exclusive for the given dataset.

#### What happens after it has identified Exclusive feature pair ?

<center><img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/062/307/original/z.png?1705487903' width=800></center>

It'll **group** the **features** into single feature
- and **create** **a new feature**

such that
- new feature will have information of both the feature


Intuitively,
- it is performing **dimensionality reduction**
    - which helps in **training** GBDT **faster**.

If you want to deep dive into it,

here a blog explaining it in detail: https://towardsdatascience.com/what-makes-lightgbm-lightning-fast-a27cf0d9785e

Let's look into how can we implement it

### Code walkthough

Documentation: https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
#Refer: https://lightgbm.readthedocs.io/en/latest/Parameters.html
import datetime as dt
gridParams = {
    'learning_rate': [0.1, 0.3, 0.5],
    'boosting_type' : ['gbdt'],
    'objective' : ['multiclass'],
    'max_depth' : [5,6,7,8],
    'colsample_bytree' : [0.5,0.7],
    'subsample' : [0.5,0.7],
    'metric':['multi_error'],
    }

clf = lgb.LGBMClassifier(num_classes=20)
random_cv = RandomizedSearchCV(clf,gridParams,verbose=3,cv=3,n_jobs = -1,n_iter=10)



In [ ]:
start = dt.datetime.now()
random_cv.fit(X_train,y_train)
end = dt.datetime.now()


Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [ ]:
res = random_cv.cv_results_

for i in range(len(res["params"])):
  print(f"Parameters:{res['params'][i]} Mean_score: {res['mean_test_score'][i]} Rank: {res['rank_test_score'][i]}")


Parameters:{'subsample': 0.7, 'objective': 'multiclass', 'metric': 'multi_error', 'max_depth': 8, 'learning_rate': 0.5, 'colsample_bytree': 0.7, 'boosting_type': 'gbdt'} Mean_score: 0.16324200913242007 Rank: 7
Parameters:{'subsample': 0.5, 'objective': 'multiclass', 'metric': 'multi_error', 'max_depth': 7, 'learning_rate': 0.3, 'colsample_bytree': 0.7, 'boosting_type': 'gbdt'} Mean_score: 0.9692415017757483 Rank: 1
Parameters:{'subsample': 0.7, 'objective': 'multiclass', 'metric': 'multi_error', 'max_depth': 8, 'learning_rate': 0.3, 'colsample_bytree': 0.7, 'boosting_type': 'gbdt'} Mean_score: 0.9681633688483003 Rank: 2
Parameters:{'subsample': 0.7, 'objective': 'multiclass', 'metric': 'multi_error', 'max_depth': 5, 'learning_rate': 0.5, 'colsample_bytree': 0.5, 'boosting_type': 'gbdt'} Mean_score: 0.1513191273465246 Rank: 9
Parameters:{'subsample': 0.5, 'objective': 'multiclass', 'metric': 'multi_error', 'max_depth': 8, 'learning_rate': 0.5, 'colsample_bytree': 0.7, 'boosting_type': '

In [ ]:
print(f"Time taken for fits : {end - start}")

Time taken for fits : 0:00:57.177334


In [ ]:
print(random_cv.best_estimator_)

LGBMClassifier(colsample_bytree=0.7, learning_rate=0.3, max_depth=7,
               metric='multi_error', num_classes=20, objective='multiclass',
               subsample=0.5)


In [ ]:
lgb = random_cv.best_estimator_

lgb.fit(X_train, y_train)

print("Model acc",lgb.score(X_test, y_test))



Model acc 0.9835150900329698


Notice that
- It only took 57 sec to train
- and its performance is comparable to XGBoost implementation

Now, that we are finished with GBDT.

Let's learn about some other ensemble techniques

## Stacking

### How does stacking works ?


Let's say we are entering a kaggle competition
- and we have a team of m members

The team decided that each individual memeber will train its own model.

So, on a give training dataset
- there will be m well hyperparameter tuned model


Note that
- all m models can be different
- i.e. M1 can be logistic regression, M2 can be Knn etc

<center><img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/062/308/original/z.png?1705487939' width=700></center>

We got the individual well trained model

#### But, how do we combine them?

Each model will give us a prediction


Using these predictions,
- we train another model
    - i.e. using predictions ($p_1, p_2, p_3 .. p_m$) as input data
    - and original target variable $y$ as target variable
    - we train a model
- This model is called meta classifier

Note:
- Instead of predictions,
    - we can take probabilties as input features for meta classifier

<center><img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/062/309/original/z.png?1705487973' width=700></center>

Note that
- Meta classifer can be any model as well (Log. reg, GBDT etc)

The prediction given by Meta classifier
- is treated as final prediction ($ŷ$)


And this whole concept is called as **stacking** of models

#### The idea looks interesting. But, why don't we use it ?

<center><img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/062/310/original/z.png?1705488004' width=700></center>

We don't use it due to its extensive **time complexity**

It takes a lot of time to fine tune M base models
- and then training the predictions on Meta classifier

Whole process becomes time consuming



However,
- it is extensively used in kaggle competition
- where the goal is to get the top score
- and not the fast models

Here's a link to the MLXTEND implementation of Stacking classifir: https://rasbt.github.io/mlxtend/user_guide/classifier/StackingClassifier/

## Cascading

As the name suggests,
- we cascade the models one after other
- i.e. chaining of models


let's see how it works

Say, we are trying to predict whether a transaction is fradulent or not



```
What will be the nature of dataset for fradulent trascation detection ?

a. Balanced dataset
b. Imbalanced dataset

Correct option: b. Imbalanced data.

```

<center><img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/062/312/original/z.png?1705488040' width=700></center>



Let the dataset be $D$ which will be imbalanced, and

- $y=1$ for fraudulent transaction
- $y =0$ for non fraudulent transaction

For a query point $x_q$,
- we will pass this point through the first model $M_1$
- Model $M_1$ will return the probability of the query point being a fraud


Based on probability, we'll split it in 2 parts:
- if the probability of ŷ  being 1 is extremely low, say $< 0.001$ then
    - we consider that as not fraudulent, let this data be $D_1$.




<center><img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/062/313/original/z.png?1705488064' width=700></center>

#### What happens to rest of the data?

The rest of the points ($D-D_1$) i.e. data with prob. > 0.001 which we are not sure about
- will be passed through the next model $M_2$
- Model $M_2$ will be more stricter i.e. it'll penalize more.

Again model $M_2$ will split into 2 parts
- non fraud (say, $P(y =1 | x_q) < 0.001$)
- fraud transac. (p > 0.001)

We can again add another model after $M_2$ which will work on same principles


<center><img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/062/314/original/z.png?1705488089' width=700></center>

<center><img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/062/315/original/z.png?1705488115' width=700></center>

#### Did you notice the structure of model?
We are cascading/chaining one model after another.

In the first model we are just removing all the genuine customers
- in second model, we are trying to find the may be fraudalent points from 2nd data set,

we contimue doing this **cascading**

Every model is trained on different datasets ($D - D_n$ )

If even after all these models, we are not sure there will be a human at last to verify the same.

Note that
- cascading is used in industry where
    -  loss associated with misclassification is high

For example:
- cancer detection
- financial domain etc